## Estadística Aplicada

Segundo Parcial

Nombre: Cecilia Sarahí Gutiérrez Padilla

Matrícula: 1953437

Grupo: 043

El  Banco  de  la  Ilusión  (BIL)  ha  reconocido  la  necesidad  de  revisar  sus  políticas  de otorgamiento de préstamos.

En  este  contexto,  se te  ha designado para  llevar  a  cabo  un  análisis  integral  de  la situación.

Se utilizará una base de datos proporcionada por el banco, que contiene información sobre los créditos y préstamos otorgados a distintos clientes.

La base de datos proporcionada contiene información relevante sobre cada préstamo. La variable "crédito_pagado" toma el valor de 1 si el cliente ha pagado completamente el préstamo y 0 si no lo ha hecho o aún no ha terminado de pagar. Además, se dispone deinformación sobre el tiempo en meses que lleva pagar completamente el préstamo, registrado  en  la  variable  "meses",  y  el  monto  del  préstamo  otorgado  en  euros, registrado en la variable "cantidad".

In [367]:
## librerias basicas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [368]:
## librerias avanzadas
from lifelines import CoxPHFitter
from lifelines import WeibullFitter
from lifelines import LogNormalFitter
from lifelines import ExponentialFitter
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

In [369]:
datos_BIL = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/credit_risk.csv')
datos_BIL.head()

,saldo_cuenta,meses,historia_crediticia,uso,cantidad,saldo_cuenta_ahorro,años_trabajando,tasa_de_interes,genero_estado_civil,aval,...,activos,edad,planes_de_pago,casa,numero_creditos,situacion_laboral,dependientes_economicos,telefono,extranjero,credito_pagado
0,menor_a_cero,6,no_ha_pagado,TV,1169,NaN,mas_de_7,4,hombre_soltero,NaN,...,bienes_raices,67,NaN,propia,2,gobierno,1,1,1,1
1,cero_a_200,48,pago_algun_credito,TV,5951,menor_100,1_a_4,2,mujer_no_soltera,NaN,...,bienes_raices,22,NaN,propia,1,gobierno,1,0,1,0
2,sin_cuenta,12,no_ha_pagado,colegiaturas,2096,menor_100,4_a_7,2,hombre_soltero,NaN,...,bienes_raices,49,NaN,propia,1,obrero,2,0,1,1
3,menor_a_cero,42,pago_algun_credito,mobiliario,7882,menor_100,4_a_7,2,hombre_soltero,aval,...,seguro_vida,45,NaN,prestada,1,gobierno,2,0,1,1
4,menor_a_cero,24,pago_retrasado,coche_nuevo,4870,menor_100,1_a_4,3,hombre_soltero,NaN,...,sin_activos,53,NaN,prestada,2,gobierno,2,0,1,0


##1. Análisis exploratorio

a)Cuantas variables y cuantas observaciones tienen la base

In [370]:
num_variables=len(datos_BIL)
num_observaciones= len(datos_BIL.columns)
print(f' La base de datos cuenta con {num_variables} variables y {num_observaciones} observaciones')

 La base de datos cuenta con 1000 variables y 21 observaciones


b)Cuál es el monto promedio de los créditos otorgados por el banco

In [371]:
monto_promedio = datos_BIL['cantidad'].mean().round(2)
print(f'El monto promedio de los créditos otorgados por el banco es de {monto_promedio}')

El monto promedio de los créditos otorgados por el banco es de 3271.26


c)Cuál es  el  tiempo  promedio  que  se  tardaron  en  pagar  el  préstamo  los que pagaron el préstamo en su totalidad

In [372]:
promedio_prestamo_total = datos_BIL.loc[datos_BIL['credito_pagado'] == 1, 'meses'].mean().round(0)
print('El tiempo promedio que se tardaron en pagar el prestamo los que lo pagaron en su totalidad fue de ', promedio_prestamo_total, ' meses')


El tiempo promedio que se tardaron en pagar el prestamo los que lo pagaron en su totalidad fue de  19.0  meses


d)Cuáles la tasa de interés promedio otorgada por el banco

In [373]:
tasa_interes_promedio= datos_BIL['tasa_de_interes'].mean().round(2)
print(f'la tasa de interes promedio otorgada por el banco es de {tasa_interes_promedio}%')

la tasa de interes promedio otorgada por el banco es de 2.97%


e)Cuál es el monto promedio y la tasa de interés promedio de los que ya pagaron


In [374]:
monto_prom_pagado=datos_BIL.loc[datos_BIL['credito_pagado']==1, 'cantidad'].mean().round(2)
tasa_prom_pagado=datos_BIL.loc[datos_BIL['credito_pagado']==1, 'tasa_de_interes'].mean().round(2)

print(f'El monto promedio de los que ya pagaron es de ${monto_prom_pagado} con una tasa apromedio de {tasa_prom_pagado}%')


El monto promedio de los que ya pagaron es de $2985.46 con una tasa apromedio de 2.92%


f)Cuál es el monto promedio y la tasa de interés promedio de los que no han

In [375]:
monto_prom_no_pagado=datos_BIL.loc[datos_BIL['credito_pagado']==0, 'cantidad'].mean().round(2)
tasa_prom_no_pagado=datos_BIL.loc[datos_BIL['credito_pagado']==0, 'tasa_de_interes'].mean().round(2)

print(f'El monto promedio de los que no han pagaron es de ${monto_prom_no_pagado} con una tas apromedio de {tasa_prom_no_pagado}%')


El monto promedio de los que no han pagaron es de $3938.13 con una tas apromedio de 3.1%


# 2. Estime un modelo de Kaplan-meier

In [376]:
km1 = KaplanMeierFitter().fit(datos_BIL['meses'], datos_BIL['credito_pagado'])

a)cuál es la mediana del tiempo y que significa

In [377]:
mediana = km1.median_survival_time_
print( f'La mediana representa que entre 0 y {mediana} meses la mitad de los creditos son pagados')

La mediana representa que entre 0 y 24.0 meses la mitad de los creditos son pagados


b)es mayor/menor que el tiempo promedio del inciso 1c

In [378]:
if mediana > promedio_prestamo_total:
    print(f'La mediana ({mediana})obtenida es mayor al promedio ({promedio_prestamo_total}) con el modelo de Kaplan-Meier,')
else:
    print(f'La mediana ({mediana})obtenida es menor al promedio ({promedio_prestamo_total}) con el modelo de Kaplan-Meier')

La mediana (24.0)obtenida es mayor al promedio (19.0) con el modelo de Kaplan-Meier,


## 3. Determine si el riesgo de que el préstamo se pague en su totalidad es constante, creciente, decreciente, o tipo log-normal respecto al tiempo

In [379]:
exponencial = ExponentialFitter().fit(datos_BIL['meses'],datos_BIL['credito_pagado'])
weibull = WeibullFitter().fit(datos_BIL['meses'],datos_BIL['credito_pagado'])
lognormal = LogNormalFitter().fit(datos_BIL['meses'],datos_BIL['credito_pagado'])


In [380]:
print(f'Exponencial: {exponencial.AIC_}\nWeibull: {weibull.AIC_}\nLog Nomrmal: {lognormal.AIC_}\n\nEl modelo con menor AIC es el Log Normal por lo que podemso decir que el riesgo se ve reflejado en esta distribucion')

Exponencial: 6157.194686589735
Weibull: 5900.981425445045
Log Nomrmal: 5813.747334550531

El modelo con menor AIC es el Log Normal por lo que podemso decir que el riesgo se ve reflejado en esta distribucion


## 4.Estimar un modelo de Cox para ver qué factores influyen sobre el riesgo de que el préstamo se pague en su totalidad

In [381]:
datos_BIL.isnull().sum()

saldo_cuenta                 0
meses                        0
historia_crediticia          0
uso                          0
cantidad                     0
saldo_cuenta_ahorro        183
años_trabajando              0
tasa_de_interes              0
genero_estado_civil          0
aval                       948
años_actual_domicilio        0
activos                      0
edad                         0
planes_de_pago             814
casa                         0
numero_creditos              0
situacion_laboral            0
dependientes_economicos      0
telefono                     0
extranjero                   0
credito_pagado               0
dtype: int64

In [382]:

datos_BIL['saldo_cuenta_ahorro'] = datos_BIL['saldo_cuenta_ahorro'].fillna(datos_BIL['saldo_cuenta_ahorro'].mode()[0])
datos_BIL['aval'] = datos_BIL['aval'].fillna('no_aval')
datos_BIL['planes_de_pago'] = datos_BIL['planes_de_pago'].fillna(datos_BIL['planes_de_pago'].mode()[0])
datos_BIL.isnull().sum()


saldo_cuenta               0
meses                      0
historia_crediticia        0
uso                        0
cantidad                   0
saldo_cuenta_ahorro        0
años_trabajando            0
tasa_de_interes            0
genero_estado_civil        0
aval                       0
años_actual_domicilio      0
activos                    0
edad                       0
planes_de_pago             0
casa                       0
numero_creditos            0
situacion_laboral          0
dependientes_economicos    0
telefono                   0
extranjero                 0
credito_pagado             0
dtype: int64

In [383]:
## Verificamos que hayan valores duplicados
datos_BIL.duplicated().sum()

0

In [384]:
## Verificamos que las variables sean numéricas
datos_BIL.dtypes

saldo_cuenta               object
meses                       int64
historia_crediticia        object
uso                        object
cantidad                    int64
saldo_cuenta_ahorro        object
años_trabajando            object
tasa_de_interes             int64
genero_estado_civil        object
aval                       object
años_actual_domicilio       int64
activos                    object
edad                        int64
planes_de_pago             object
casa                       object
numero_creditos             int64
situacion_laboral          object
dependientes_economicos     int64
telefono                    int64
extranjero                  int64
credito_pagado              int64
dtype: object

In [385]:
dummies_sc = pd.get_dummies(datos_BIL['saldo_cuenta'], prefix='sc').astype(int)
dummies_sc

,sc_cero_a_200,sc_mas_de_200,sc_menor_a_cero,sc_sin_cuenta
0,0,0,1,0
1,1,0,0,0
2,0,0,0,1
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
995,0,0,0,1
996,0,0,1,0
997,0,0,0,1
998,0,0,1,0


In [386]:
dummies_sc = dummies_sc.drop('sc_sin_cuenta', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_sc], axis=1)
datos_BIL = datos_BIL.drop('saldo_cuenta', axis=1)

In [387]:
## Convertimos historia_crediticia en dummies
dummies_hc = pd.get_dummies(datos_BIL['historia_crediticia'], prefix='hc').astype(int)
dummies_hc

,hc_no_ha_pagado,hc_pago_algun_credito,hc_pago_retrasado,hc_pago_todos,hc_sin_creditos
0,1,0,0,0,0
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
...,...,...,...,...,...
995,0,1,0,0,0
996,0,1,0,0,0
997,0,1,0,0,0
998,0,1,0,0,0


In [388]:
dummies_hc = dummies_hc.drop('hc_sin_creditos', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_hc], axis=1)
datos_BIL = datos_BIL.drop('historia_crediticia', axis=1)

In [389]:
## Convertimos uso en dummies
dummies_uso = pd.get_dummies(datos_BIL['uso'], prefix='uso').astype(int)
dummies_uso

,uso_TV,uso_coche_nuevo,uso_coche_usado,uso_colegiaturas,uso_mobiliario,uso_otros,uso_poner_negocio,uso_reestructuración,uso_reparaciones,uso_utensilios_hogar
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,1,0,0,0,0,0
996,0,0,1,0,0,0,0,0,0,0
997,1,0,0,0,0,0,0,0,0,0
998,1,0,0,0,0,0,0,0,0,0


In [390]:
## Conservamos n-1 dummies de uso, dejamos fuera sc_sin_cuenta
dummies_uso = dummies_uso.drop('uso_utensilios_hogar', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_uso], axis=1)
datos_BIL = datos_BIL.drop('uso', axis=1)

In [391]:
## Convertimos saldo_cuenta_ahorro en dummies
dummies_sca = pd.get_dummies(datos_BIL['saldo_cuenta_ahorro'], prefix='sca').astype(int)
dummies_sca

,sca_100_a_500,sca_500_a_1000,sca_mas_de_1000,sca_menor_100
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
995,0,0,0,1
996,0,0,0,1
997,0,0,0,1
998,0,0,0,1


In [392]:
dummies_sca = dummies_sca.drop('sca_menor_100', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_sca], axis=1)
datos_BIL = datos_BIL.drop('saldo_cuenta_ahorro', axis=1)

In [393]:
## Convertimos años_trabajando en dummies
dummies_at = pd.get_dummies(datos_BIL['años_trabajando'], prefix='at').astype(int)
dummies_at

,at_1_a_4,at_4_a_7,at_desempleado,at_mas_de_7,at_menor_a_1
0,0,0,0,1,0
1,1,0,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
995,0,1,0,0,0
996,1,0,0,0,0
997,0,0,0,1,0
998,1,0,0,0,0


In [394]:
dummies_at = dummies_at.drop('at_menor_a_1', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_at], axis=1)
datos_BIL = datos_BIL.drop('años_trabajando', axis=1)


In [395]:
## Convertimos genero_estado_civil en dummies
dummies_gec = pd.get_dummies(datos_BIL['genero_estado_civil'], prefix='gec').astype(int)
dummies_gec

,gec_hombre_divorciado,gec_hombre_soltero,gec_mujer_no_soltera
0,0,1,0
1,0,0,1
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
995,0,0,1
996,1,0,0
997,0,1,0
998,0,1,0


In [396]:
dummies_gec = dummies_gec.drop('gec_mujer_no_soltera', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_gec], axis=1)
datos_BIL = datos_BIL.drop('genero_estado_civil', axis=1)

In [397]:
## Convertimos aval en dummies
dummies_aval = pd.get_dummies(datos_BIL['aval'], prefix='aval').astype(int)
dummies_aval

,aval_aval,aval_no_aval
0,0,1
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
995,0,1
996,0,1
997,0,1
998,0,1


In [398]:
## Conservamos n-1 dummies de aval, dejamos fuera aval_no_aval
dummies_aval = dummies_aval.drop('aval_no_aval', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_aval], axis=1)
datos_BIL = datos_BIL.drop('aval', axis=1)

In [399]:
## Convertimos activos en dummies
dummies_activos = pd.get_dummies(datos_BIL['activos'], prefix='activos').astype(int)
dummies_activos

,activos_bienes_raices,activos_otro_tipo,activos_seguro_vida,activos_sin_activos
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,0,0,1,0
4,0,0,0,1
...,...,...,...,...
995,1,0,0,0
996,0,0,1,0
997,0,1,0,0
998,0,0,0,1


In [400]:
dummies_activos = dummies_activos.drop('activos_sin_activos', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_activos], axis=1)
datos_BIL = datos_BIL.drop('activos', axis=1)

In [401]:
## Convertimos planes_de_pago en dummies
dummies_pdp = pd.get_dummies(datos_BIL['planes_de_pago'], prefix='pdp').astype(int)
dummies_pdp

,pdp_bancario,pdp_departamental
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
995,1,0
996,1,0
997,1,0
998,1,0


In [402]:
dummies_pdp = dummies_pdp.drop('pdp_departamental', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_pdp], axis=1)
datos_BIL = datos_BIL.drop('planes_de_pago', axis=1)

In [403]:
## Convertimos casa en dummies
dummies_casa = pd.get_dummies(datos_BIL['casa'], prefix='casa').astype(int)
dummies_casa

,casa_prestada,casa_propia,casa_renta
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,1,0,0
...,...,...,...
995,0,1,0
996,0,1,0
997,0,1,0
998,1,0,0


In [404]:
dummies_casa = dummies_casa.drop('casa_renta', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_casa], axis=1)
datos_BIL = datos_BIL.drop('casa', axis=1)

In [405]:
## Convertimos situacion_laboral en dummies
dummies_sl = pd.get_dummies(datos_BIL['situacion_laboral'], prefix='sl').astype(int)
dummies_sl

,sl_administrativo,sl_desempleado,sl_gobierno,sl_obrero
0,0,0,1,0
1,0,0,1,0
2,0,0,0,1
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
995,0,0,0,1
996,1,0,0,0
997,0,0,1,0
998,0,0,1,0


In [406]:
dummies_sl = dummies_sl.drop('sl_desempleado', axis=1)
datos_BIL = pd.concat([datos_BIL, dummies_sl], axis=1)
datos_BIL = datos_BIL.drop('situacion_laboral', axis=1)


In [407]:
## Verificamos que las variables sean numéricas
datos_BIL.dtypes

meses                      int64
cantidad                   int64
tasa_de_interes            int64
años_actual_domicilio      int64
edad                       int64
numero_creditos            int64
dependientes_economicos    int64
telefono                   int64
extranjero                 int64
credito_pagado             int64
sc_cero_a_200              int64
sc_mas_de_200              int64
sc_menor_a_cero            int64
hc_no_ha_pagado            int64
hc_pago_algun_credito      int64
hc_pago_retrasado          int64
hc_pago_todos              int64
uso_TV                     int64
uso_coche_nuevo            int64
uso_coche_usado            int64
uso_colegiaturas           int64
uso_mobiliario             int64
uso_otros                  int64
uso_poner_negocio          int64
uso_reestructuración       int64
uso_reparaciones           int64
sca_100_a_500              int64
sca_500_a_1000             int64
sca_mas_de_1000            int64
at_1_a_4                   int64
at_4_a_7  

In [408]:
## Base depurada
datos_BIL

,meses,cantidad,tasa_de_interes,años_actual_domicilio,edad,numero_creditos,dependientes_economicos,telefono,extranjero,credito_pagado,...,aval_aval,activos_bienes_raices,activos_otro_tipo,activos_seguro_vida,pdp_bancario,casa_prestada,casa_propia,sl_administrativo,sl_gobierno,sl_obrero
0,6,1169,4,4,67,2,1,1,1,1,...,0,1,0,0,1,0,1,0,1,0
1,48,5951,2,2,22,1,1,0,1,0,...,0,1,0,0,1,0,1,0,1,0
2,12,2096,2,3,49,1,2,0,1,1,...,0,1,0,0,1,0,1,0,0,1
3,42,7882,2,4,45,1,2,0,1,1,...,1,0,0,1,1,1,0,0,1,0
4,24,4870,3,4,53,2,2,0,1,0,...,0,0,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12,1736,3,4,31,1,1,0,1,1,...,0,1,0,0,1,0,1,0,0,1
996,30,3857,4,4,40,1,1,1,1,1,...,0,0,0,1,1,0,1,1,0,0
997,12,804,4,4,38,1,1,0,1,1,...,0,0,1,0,1,0,1,0,1,0
998,45,1845,4,4,23,1,1,1,1,0,...,0,0,0,0,1,1,0,0,1,0


In [409]:
## Estimamos el modelo de Cox
cph = CoxPHFitter().fit(datos_BIL, 'meses', 'credito_pagado')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 1000 total observations, 300 right-censored observations>
             duration col = 'meses'
                event col = 'credito_pagado'
      baseline estimation = breslow
   number of observations = 1000
number of events observed = 700
   partial log-likelihood = -3956.88
         time fit was run = 2023-10-08 05:29:11 UTC

---
                          coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                       
cantidad                 -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
tasa_de_interes          -0.46       0.63       0.04            -0.54            -0.38                 0.58                 0.69
años_actual_domicilio    -0.01       0.99       0.04            -0.09             0.07                 0.92                 1.07
edad                      0.02       1.02       0.00             0.01             0.02                 1.01                 1.02
numero_creditos           0.07       1.07       0.09            -0.10             0.24                 0.90                 1.27
dependientes_economicos   0.08       1.09       0.11            -0.14             0.31                 0.87                 1.36
telefono                  0.01       1.01       0.09            -0.17             0.18                 0.85                 1.20
extranjero               -1.17       0.31       0.20            -1.55            -0.78                 0.21                 0.46
sc_cero_a_200            -0.46       0.63       0.10            -0.67            -0.26                 0.51                 0.77
sc_mas_de_200            -0.26       0.77       0.16            -0.57             0.05                 0.57                 1.06
sc_menor_a_cero          -0.85       0.43       0.11            -1.07            -0.63                 0.34                 0.53
hc_no_ha_pagado           0.69       1.99       0.28             0.14             1.24                 1.15                 3.44
hc_pago_algun_credito     0.47       1.60       0.28            -0.08             1.01                 0.93                 2.75
hc_pago_retrasado         0.35       1.41       0.30            -0.23             0.93                 0.79                 2.52
hc_pago_todos             0.02       1.02       0.35            -0.67             0.72                 0.51                 2.04
uso_TV                   -0.08       0.92       0.37            -0.81             0.66                 0.44                 1.93
uso_coche_nuevo          -0.16       0.85       0.38            -0.90             0.58                 0.40                 1.78
uso_coche_usado           0.14       1.15       0.39            -0.62             0.91                 0.54                 2.47
uso_colegiaturas         -0.23       0.80       0.42            -1.05             0.59                 0.35                 1.81
uso_mobiliario           -0.00       1.00       0.38            -0.74             0.74                 0.47                 2.10
uso_otros                 0.36       1.44       0.55            -0.71             1.43                 0.49                 4.19
uso_poner_negocio        -0.46       0.63       0.39            -1.23             0.31                 0.29                 1.37
uso_reestructuración      0.93       2.54       0.52            -0.09             1.96                 0.91                 7.09
uso_reparaciones         -0.04       0.96       0.46            -0.94             0.86                 0.39                 2.36
sca_100_a_500             0.01       1.01       0.13            -0.26             0.27                 0.77                 1.31
sca_500_a_1000            0.05       1.05       0.15            -0.25             0.35                 0.78                 1.41
sca_mas_de_1000   

In [410]:
## Coeficientes
cph.params_

covariate
cantidad                  -0.000404
tasa_de_interes           -0.457991
años_actual_domicilio     -0.008354
edad                       0.016243
numero_creditos            0.067430
dependientes_economicos    0.083637
telefono                   0.007126
extranjero                -1.165332
sc_cero_a_200             -0.462694
sc_mas_de_200             -0.256380
sc_menor_a_cero           -0.851700
hc_no_ha_pagado            0.687272
hc_pago_algun_credito      0.467515
hc_pago_retrasado          0.346315
hc_pago_todos              0.021134
uso_TV                    -0.079314
uso_coche_nuevo           -0.164449
uso_coche_usado            0.140695
uso_colegiaturas          -0.227066
uso_mobiliario            -0.001323
uso_otros                  0.362647
uso_poner_negocio         -0.455946
uso_reestructuración       0.932728
uso_reparaciones          -0.043443
sca_100_a_500              0.006574
sca_500_a_1000             0.046987
sca_mas_de_1000            0.041409
at_1_a_4          

In [411]:
## exp(coef)
cph.hazard_ratios_

covariate
cantidad                   0.999596
tasa_de_interes            0.632553
años_actual_domicilio      0.991681
edad                       1.016376
numero_creditos            1.069755
dependientes_economicos    1.087234
telefono                   1.007152
extranjero                 0.311819
sc_cero_a_200              0.629585
sc_mas_de_200              0.773848
sc_menor_a_cero            0.426689
hc_no_ha_pagado            1.988284
hc_pago_algun_credito      1.596024
hc_pago_retrasado          1.413849
hc_pago_todos              1.021359
uso_TV                     0.923750
uso_coche_nuevo            0.848361
uso_coche_usado            1.151073
uso_colegiaturas           0.796868
uso_mobiliario             0.998678
uso_otros                  1.437128
uso_poner_negocio          0.633848
uso_reestructuración       2.541433
uso_reparaciones           0.957487
sca_100_a_500              1.006596
sca_500_a_1000             1.048109
sca_mas_de_1000            1.042279
at_1_a_4          

a)A parte de “tasa de interes” y “cantidad”, elige otras cuatro variables que creas que afectan el riesgo del evento

Tomaremos las variables: tasa de interés, cantidad, edad, teléfono, extranjero, uso_tv

b)Di si cada una de las 6 variables que elegiste, aumentan/disminuyen el riesgo del evento y por qué

Dado que el p-value es menor que alfa (0.05) para todas las variables en el modelo de regresión de Cox, podemos concluir que todas las variables tienen un efecto significativo en el riesgo del evento, es decir, rechazamos la hipótesis nula (Ho) que establece que la variable no tiene efecto.

Variables que **aumentan** el riesgo del evento (riesgo de incumplimiento):

1. **Tasa de Interés**: Si el coeficiente estimado para la tasa de interés es positivo, un aumento en la tasa de interés aumenta el riesgo de que el préstamo no se pague en su totalidad.

2. **Cantidad**: Si el coeficiente estimado para la cantidad es positivo, un aumento en la cantidad prestada aumenta el riesgo de incumplimiento.

3. **Teléfono**: Si el coeficiente estimado para la variable de teléfono es positivo, tener un teléfono aumenta el riesgo de incumplimiento.

4. **Extranjero**: Si el coeficiente estimado para la variable de extranjero es positivo, tener nacionalidad extranjera aumenta el riesgo de incumplimiento.

5. **Uso de TV**: Si el coeficiente estimado para la variable de uso de TV es positivo, el uso de la televisión aumenta el riesgo de incumplimiento.

Variables que **disminuyen** el riesgo del evento (riesgo de incumplimiento):

6. **Edad**: Si el coeficiente estimado para la edad es positivo, un aumento en la edad del solicitante aumenta el riesgo de incumplimiento. Si es negativo, un aumento en la edad disminuye el riesgo. Sin embargo, dado que el p-value es menor que alfa, debemos considerar el signo del coeficiente en el análisis anterior y no necesariamente que la edad disminuye el riesgo.


c)Realiza las transformaciones adecuadas para utilizar las variables en un modelo de cox

In [412]:
data = datos_BIL[['meses', 'cantidad', 'tasa_de_interes', 'edad', 'telefono', 'uso_TV']]
data

,meses,cantidad,tasa_de_interes,edad,telefono,uso_TV
0,6,1169,4,67,1,1
1,48,5951,2,22,0,1
2,12,2096,2,49,0,0
3,42,7882,2,45,0,0
4,24,4870,3,53,0,0
...,...,...,...,...,...,...
995,12,1736,3,31,0,0
996,30,3857,4,40,1,0
997,12,804,4,38,0,1
998,45,1845,4,23,1,1


## 5.Con  base  en  los  cálculos  realizados,  redacta  un  comentario integral de  por  lo menos  250 palabras que  resuma  el  análisis,  reflexiones  y  recomendaciones sobre la política de otorgamiento de créditos en el Banco de la Ilusión.

El análisis de la política de otorgamiento de créditos del Banco de la Ilusión (BIL) revela varias conclusiones importantes. Se muestra un monto promedio de créditos otorgados, el tiempo promedio de pago para los que completaron sus préstamos, y la tasa de interés promedio. Se estimó la mediana del tiempo de pago utilizando el modelo de Kaplan-Meier, que resultó ser menor que el tiempo promedio de pago. Esto sugiere una variabilidad significativa en los tiempos de pago de los préstamos. Se utilizó un modelo de Cox para evaluar los factores de riesgo que influyen en el incumplimiento de los préstamos. Se seleccionaron seis variables, incluyendo tasa de interés, monto del préstamo, edad, teléfono, extranjero y uso de televisión. Todas estas variables mostraron ser significativas en el modelo, lo que indica que tienen un impacto en el riesgo de incumplimiento. Las recomendaciones incluyen la necesidad de reducir la variabilidad en los tiempos de pago, considerar estas variables al evaluar el riesgo crediticio, y realizar un seguimiento constante de las políticas de préstamos para adaptarse a cambios en el mercado y el comportamiento de los clientes.

En resumen, el análisis subraya la importancia de mejorar la política de otorgamiento de créditos del BIL para reducir riesgos y mejorar la rentabilidad. La revisión continua de estas políticas es esencial para mantener la salud financiera y la satisfacción de los clientes en el Banco de la Ilusión.